In [1]:
import polars as pl
from hashlib import sha256
import os
import json
from urllib.parse import urlparse, parse_qs
pl.Config.set_fmt_str_lengths(400)

polars.config.Config

In [2]:
DATA_PATH = os.getenv("DATA_PATH", "")

In [4]:
df = pl.read_parquet(f"{DATA_PATH}/interim/aggregated_links_statistics.parquet")
casts_lazy_df = pl.scan_parquet(f"{DATA_PATH}/raw/farcaster-casts-0-1730134800.parquet")

In [5]:
df

created_at,updated_at,custody_address,registered_at,fid,verifications_count,following_count,followers_count,likes_count,recasts_count,total_casts_count,reply_casts_count,casts_count
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,i64,i64,i64,i64,i64,i64,i64,i64
2023-10-13 15:07:21.731280,2024-06-05 18:09:02.020,"b""\xb3\x97\x15\x8b\\xb5\x02\x03\xa5\x91j\x07\x12U)\xb1I\xca\xe3<""",2023-11-07 20:38:19 UTC,42796,0,0,1,0,0,0,0,0
2023-10-13 15:07:21.732119,2024-06-05 18:09:02.024,"b""\xed\x93\xe1\xef\x7f+\xef\x9c\x94\x9a\xa1\xba\xca\xa8\x9c\x10\xae\xae\x1b\xac""",2023-11-07 20:38:19 UTC,42797,0,0,2,0,0,0,0,0
2023-10-13 15:07:21.734257,2024-06-05 18:09:02.025,"b""\xf6\x9bv\x99\xd2/\xb1\x87z6\x10\xef\x81\xe9{\x05\x20\x03\xc5\xac""",2023-11-07 20:38:19 UTC,42798,0,0,1,0,0,0,0,0
2023-10-13 15:32:07.129182,2024-06-05 18:10:59.760,"b""\xa1-f\x16\x8e\x88\x18\xe8-hS\x1d_c\x09\xe6\x07/\xc8\xf5""",2023-11-07 20:49:51 UTC,64737,0,0,1,0,0,0,0,0
2023-10-13 15:53:45.836242,2024-06-05 18:12:26.055,"b""\x0f\xc9\x11\xd1S\xbd\xb0*<p;\xd6RG\xb2\x80\xd8\xe6Y\x85""",2023-11-07 20:57:45 UTC,80519,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…
2024-10-28 16:51:40.194175,2024-10-28 16:51:40.194175,"b""\xbdwP\xbbb\xeb\xf0\xd4\xa3o\xfc\xb8\x02\xd4#\x09\x956VL""",2024-10-28 16:51:31 UTC,871144,0,0,0,0,0,0,0,0
2024-10-28 16:52:04.477017,2024-10-28 16:52:04.477017,"b""jDdHLS\xa5""\x1e\x00v\xb4\xa6\xab\x19\xb3I?|&""",2024-10-28 16:51:57 UTC,871145,0,0,0,1,1,0,0,0
2024-10-28 16:54:50.861109,2024-10-28 16:54:50.861109,"b""vR/G\xb2w\xf3\x0d\x82\xfc\x92\x01\x97\xed\xcd1<S\xfc\x93""",2024-10-28 16:54:43 UTC,871146,0,0,0,0,0,0,0,0


In [6]:
casts_count_lazy = (
    casts_lazy_df
    .filter(pl.col("deleted_at").is_null())  # Filter out deleted casts
    .group_by("fid")
    .agg(pl.count("id").alias("casts_count"))  # Aggregate to count casts per fid
)

In [6]:
# https://warpcast.com/eljuniordiaz.eth/0xf612041a
# https://warpcast.com/falleccypollne/0x04253bc9

# https://warpcast.com/sayangel/0x8cfc0da0

In [7]:
# https://warpcast.com/botornot
BOT_OR_NOT_FID = 480626
bot_or_not_casts_lazy = casts_lazy_df.filter(pl.col("fid") == BOT_OR_NOT_FID)
bot_or_not_casts_df = bot_or_not_casts_lazy.collect()
bot_or_not_casts_df.write_parquet(f"{DATA_PATH}/interim/bot_or_not_casts.parquet")
# bot_or_not_casts_df = pl.read_parquet(f"{DATA_PATH}/interim/bot_or_not_casts.parquet")

In [8]:
bot_or_not_casts_df

created_at,updated_at,deleted_at,timestamp,fid,hash,parent_hash,parent_fid,parent_url,text,embeds,mentions,mentions_positions,root_parent_hash,root_parent_url,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,binary,i64,str,str,str,str,str,binary,str,i64
2024-04-14 09:35:38.778506,2024-04-14 09:35:38.778506,null,2024-04-14 09:35:39,480626,"b""\x0f\x9f\xe6\xa6\x0f\x9e\x1b4ij\xf5\x10\x1f\x80+\x09\xcf}\xe6\x19""",null,null,null,"""Bot or Not? Someone checked : Bot or low effort casts: squares, numbers, random""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames?fid=374893""}]""","""[374893]""","""[28]""","b""\x0f\x9f\xe6\xa6\x0f\x9e\x1b4ij\xf5\x10\x1f\x80+\x09\xcf}\xe6\x19""",null,2037557994
2024-04-14 09:35:42.438214,2024-04-14 09:35:42.438214,null,2024-04-14 09:35:42,480626,"b""Ft\x9c({\xd8\xc1\xc6<[\xfde\x1a\x9a\xdc\xa1\x11\xb6CX""",null,null,null,"""Bot or Not? Someone checked : This user casts like a bot.""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames?fid=374893""}]""","""[374893]""","""[28]""","b""Ft\x9c({\xd8\xc1\xc6<[\xfde\x1a\x9a\xdc\xa1\x11\xb6CX""",null,2037558093
2024-04-14 09:35:51.097913,2024-04-14 09:35:51.097913,null,2024-04-14 09:35:51,480626,"b""E\x1f\x89\x1ed\xbe\xdd)\xfc\xc6\xa68\x1e\xad\xfe\x1b5\xd7\xe0\xff""",null,null,null,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic, vapid""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames?fid=246611""}]""","""[246611]""","""[28]""","b""E\x1f\x89\x1ed\xbe\xdd)\xfc\xc6\xa68\x1e\xad\xfe\x1b5\xd7\xe0\xff""",null,2037558310
2024-04-14 09:35:55.805754,2024-04-14 09:35:55.805754,null,2024-04-14 09:35:56,480626,"b""\x16\x94\xba\x8c\xfeI6w\xd7\xc4\x15\xd6\xfa\xdc=h\xf9\xfe]\x97""",null,null,null,"""Bot or Not? Someone checked : This user replies like a bot: meat, degen, generic""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames?fid=378852""}]""","""[378852]""","""[28]""","b""\x16\x94\xba\x8c\xfeI6w\xd7\xc4\x15\xd6\xfa\xdc=h\xf9\xfe]\x97""",null,2037558414
2024-04-14 09:35:58.416466,2024-04-14 09:35:58.416466,null,2024-04-14 09:35:58,480626,"b""/\xaf\x02\xfac\x0d\x01\xf1\x94\xeb&i\x20\xb1L\xdfu.$U""",null,null,null,"""Bot or Not? Someone checked : This user replies like a bot: generic, shallow""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames?fid=435074""}]""","""[435074]""","""[28]""","b""/\xaf\x02\xfac\x0d\x01\xf1\x94\xeb&i\x20\xb1L\xdfu.$U""",null,2037558496
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-10-28 03:27:28.063779,2024-10-28 03:27:28.063779,null,2024-10-28 03:27:28,480626,"b""\x01\x02\xbd5{\x90\x89\x9eAk\x910:<u\x13Z\x8d\x99\xad""",null,null,null,"""Bot or Not? Someone checked : Not a bot. Casts: ramen, /wabisabi, poetry Want to check someone else? Install the cast action below.""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames/responses/summary?fid=222540&username=bashobits""}]""","""[222540]""","""[28]""","b""\x01\x02\xbd5{\x90\x89\x9eAk\x910:<u\x13Z\x8d\x99\xad""",null,5874276022
2024-10-28 08:19:07.551290,2024-10-28 08:19:07.551290,null,2024-10-28 08:19:08,480626,"b""\x98#a\xa1\xe9tO\xfcp\x0f\xf2\xe2\xda\xd8\xa8\x0aBM\x8a\x14""",null,null,null,"""Bot or Not? Someone checked : Not a bot. Casts: inspiration, poetry, gm frens Want to check someone else? Install the cast action below.""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames/responses/summary?fid=543177&username=trpplffct""}]""","""[543177]""","""[28]""","b""\x98#a\xa1\xe9tO\xfcp\x0f\xf2\xe2\xda\xd8\xa8\x0aBM\x8a\x14""",null,5874382370
2024-10-28 10:47:44.707538,2024-10-28 10:47:44.707538,null,2024-10-28 10:47:45,480626,"b""!\x7f\x82\xfd\x0e\xf8\x03\x81\xc2\xc5\xf6\x95\xe9\xedA\xcc\x90\xfb\xfd\x0e""",null,null,null,"""Bot or Not? Someone checked : Not a bot. Casts: $MOXIE, Farcaster, crypto projects Want to check someone else? Install the cast action below.""","""[{""url"": ""https://cast-action-bot-or-not.vercel.app/api/frames/res

In [9]:
bots_casts_df = bot_or_not_casts_df.filter(~pl.col("text").str.contains(r"(?i)not a bot\."))
# bots_casts_df = bots_casts_df.filter(~pl.col("text").str.contains("Bot or Not? Someone checked : Casts: "))
# bots_casts_df = bots_casts_df.filter(~pl.col("text").str.contains("Bot or Not? Someone checked: Casts: "))
bots_casts_df = bots_casts_df.filter(~pl.col("text").str.contains("Hi ! You have reached your Bot or Not free limit. "))
bots_casts_df = bots_casts_df.filter((pl.col("text").str.contains(r"(?i)like a bot") | pl.col("text").str.contains(r"(?i)bot or low")))

bots_casts_df = bots_casts_df.with_columns(
    pl.col("hash").map_elements(lambda x: "0x" + x.hex(), return_dtype=pl.Utf8).alias("cast_hash")
)
# bots_casts_df
# unique_casts_df = bots_casts_df.unique(subset=["text"])
# sampled_casts_df = unique_casts_df.select(["text","cast_hash" ]).sample(10)
# print(bots_casts_df.shape)
# bots_casts_df = bots_casts_df.unique
# print(bots_casts_df.unique(subset=["mentions"]).shape)

bots_casts_df = bots_casts_df.unique(subset=["mentions"])

In [10]:
not_bots_df = bot_or_not_casts_df.filter(pl.col("text").str.contains(r"(?i)not a bot\."))

In [11]:
# build fids
def transform_mentions(row):
    mentions = json.loads(row['mentions'])
    embeds = json.loads(row['embeds'])

    if mentions:
        return int(mentions[0])

    if embeds:
        url = embeds[0]["url"]
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        if 'fid' in query_params:
            return int(query_params['fid'][0])

    return None


In [12]:
# testing = not_bots_df.filter(pl.col("mentions") == "[]")
# testing = testing.with_columns(
#     pl.struct(["mentions", "embeds"]).map_elements(transform_mentions, return_dtype=pl.Int64).alias("fid_new")
# )

In [12]:
bots_casts_df = bots_casts_df.with_columns(
    pl.struct(["mentions", "embeds"]).map_elements(transform_mentions, return_dtype=pl.Int64).alias("fid")
)
not_bots_df = not_bots_df.with_columns(
    pl.struct(["mentions", "embeds"]).map_elements(transform_mentions, return_dtype=pl.Int64).alias("fid")
)

In [13]:
bots_casts_df = bots_casts_df.unique(subset=['fid'])

In [14]:
not_bots_df = not_bots_df.unique(subset=['fid'])

In [15]:
not_bots_df = not_bots_df.with_columns(
    pl.lit(False).alias('bot')
)

bots_casts_df = bots_casts_df.with_columns(
    pl.lit(True).alias('bot')
)

In [16]:
not_bots_df = not_bots_df.select(['fid', 'bot', 'text'])
bots_casts_df = bots_casts_df.select(['fid', 'bot', 'text'])

In [17]:
bots_casts_df #.unique(subset=['fid'])

fid,bot,text
i64,bool,str
16920,true,"""Bot or Not? Someone checked : This user casts like a bot: gm, degen, events, metaverse Want to check someone else? Install the cast action below."""
482904,true,"""Bot or Not? Someone checked : This user casts like a bot: promo garbage fake tips shitcast Want to check someone else? Install the cast action below."""
259313,true,"""Bot or Not? Someone checked : This user replies like a bot: spammy, repetitive"""
393940,true,"""Bot or Not? Someone checked : This user casts like a bot. Want to check someone else? Install the cast action below."""
15351,true,"""Bot or Not? Someone checked : This user casts like a bot."""
…,…,…
300929,true,"""Bot or Not? Someone checked : This user replies like a bot: emoji, random"""
471361,true,"""Bot or Not? Someone checked : This user casts like a bot: nft,ham,food,crypto,airdrop,price Want to check someone else? Install the cast action below."""
450383,true,"""Bot or Not? Someone checked : This user replies like a bot: spammy, repetitive, generic Want to check someone else? Install the cast action below."""


In [18]:
combined_df = pl.concat([not_bots_df, bots_casts_df])

In [19]:
combined_df

fid,bot,text
i64,bool,str
332182,false,"""Bot or Not? Someone checked : Likely not a bot. Casts: trashion, photography, AI"""
238841,false,"""Bot or Not? Someone checked : Not a bot. Casts: steps, degen, milady"""
405599,false,"""Bot or Not? Someone checked : Not a bot. Casts: degen, mint, giveaways"""
15211,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: based, builder, safety, gm, gn Want to check someone else? Install the cast action below."""
378696,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic, repetitive"""
…,…,…
300929,true,"""Bot or Not? Someone checked : This user replies like a bot: emoji, random"""
471361,true,"""Bot or Not? Someone checked : This user casts like a bot: nft,ham,food,crypto,airdrop,price Want to check someone else? Install the cast action below."""
450383,true,"""Bot or Not? Someone checked : This user replies like a bot: spammy, repetitive, generic Want to check someone else? Install the cast action below."""


In [279]:
# combined_df.write_parquet(f"{DATA_PATH}/bot_or_not_checks.parquet")

In [24]:
combined_df

fid,bot,text
i64,bool,str
350129,false,"""Bot or Not? Someone checked : Likely not a bot. Casts: social, games, tips"""
274,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: shilling $DEGEN"""
243560,false,"""Bot or Not? Someone checked : Not a bot. Casts: drakula spammer"""
421161,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: pancakes, freediving"""
379765,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: short, generic, boring casts Want to check someone else? Install the cast action below."""
…,…,…
363848,true,"""Bot or Not? Someone checked : This user casts like a bot. Want to check someone else? Install the cast action below."""
11004,true,"""Bot or Not? Someone checked : Bot or low effort casts: generic, repetitive, empty, spam Want to check someone else? Install the cast action below."""
416302,true,"""Bot or Not? Someone checked : This user casts like a bot."""


In [27]:
combined_df.unique(subset=['fid'])

fid,bot,text
i64,bool,str
256032,true,"""Bot or Not? Someone checked : This user casts like a bot."""
505070,true,"""Bot or Not? Someone checked : This user casts like a bot: korean, generic, ham, positivity Want to check someone else? Install the cast action below."""
386711,true,"""Bot or Not? Someone checked : Bot or low effort casts: emojis, short"""
424591,true,"""Bot or Not? Someone checked : This user replies like a bot: repetitive, spammy degen casts Want to check someone else? Install the cast action below."""
254591,false,"""Bot or Not? Someone checked : Likely not a bot. Casts: crypto, bullish sentiment, design Want to check someone else? Install the cast action below."""
…,…,…
402017,false,"""Bot or Not? Someone checked : Likely not a bot. Casts: korean crypto"""
428625,true,"""Bot or Not? Someone checked : This user casts like a bot. Want to check someone else? Install the cast action below."""
401621,true,"""Bot or Not? Someone checked : Bot or low effort casts: repetitive, korean, hype, casts Want to check someone else? Install the cast action below."""


In [21]:
duplicateds = (
    combined_df.group_by(['fid'])
    .agg(pl.count())
    .filter(pl.col("count") > 1)  # Filter groups where count > 1 (duplicates)
)

duplicateds
# Step 2: Join the duplicate groups back to the original DataFrame
# This retrieves all versions (original + duplicates)
# duplicates_df = df.join(duplicate_groups.drop("count"), on=subset_cols, how="inner")

/var/folders/k2/v9m0s0cn02j1r54h8qh_tf200000gn/T/ipykernel_86866/172828407.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count())


fid,count
i64,u32
264690,2
407433,2
376106,2
339009,2
363387,2
…,…
423117,2
278897,2
405926,2


In [22]:
duplicated_fids = duplicateds.select(['fid'])

In [24]:
fnames_df = pl.read_parquet(f"{DATA_PATH}/raw/farcaster-fnames-0-1730134800.parquet")

In [36]:
# combined_df.filter(pl.col('fid')==291366)

fid,bot,text
i64,bool,str
291366,false,"""Bot or Not? Someone checked : Likely not a bot. Casts: crypto, ai, nft, airdrops, shitcasts Want to check someone else? Install the cast action below."""
291366,true,"""Bot or Not? Someone checked : This user replies like a bot: generic, repetitive"""


In [37]:
# fnames_df.filter(pl.col('fid')==291366)

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-02-05 12:24:17.902048,2024-06-07 22:07:32.665,"b""\xd8\x88y\x17@-\x8b\xea\xc5\xbe\xaa55\xa6N\xe5_)\x94D""",null,291366,null,"""kon102"""


In [25]:
without_duplicates = combined_df.filter(~pl.col('fid').is_in(duplicated_fids.to_series()))

In [26]:
without_duplicates.write_parquet(f"{DATA_PATH}/interim/bot_or_not_checks.parquet")